In [279]:
import networkx as nx
import igraph as ig
import numpy as np
import scipy.stats as ss
import pandas as pd
from collections import Counter

In [354]:
class distance:
    
    """
    distance class
    args: 
        -reference graph 'G0' (igraph object)
        -proposal graph 'G_prop' generated from model (igraph object)
        -function 'measure' taking two igraph objects and returning a scalar 
        -'**measure_kwargs' are optional keyword args that may apply to 'measure'
    """
    
    def __init__(self,G0,measure,**measure_kwargs):
        
        """
        distance class attributes
        """
        
        self.measure = measure
        self.G0 = G0
        self.measure_kwargs = measure_kwargs
        
    def evaluate_unnormalized(self,G_prop):
        
        """
        evaluate unnormalized distance between G0 and igraph object G_prop
        """
        self.G_prop = G_prop
        
        return self.measure(self.G0,self.G_prop,**self.measure_kwargs)
    
    def evaluate_normalized(self,G_prop,num_nulls,null_generator,**null_kwargs): 
        
        """
        evaluate normalized distance between G0 and igraph object G_prop
        args:
            -'num_nulls' is number of null graphs to generate to compute z-score
            -'null_generator' is function that takes in a reference graph G0 and outputs an igraph object that is randomized
                    based on some property of G0 and some model/parameters included in **null_kwargs
        """
        
        null_distance_vals = [self.measure(self.G0,null_generator(self.G0,**null_kwargs),**self.measure_kwargs)\
                              for _ in range(num_nulls)]
        null_mean,null_std = np.mean(null_distance_vals),np.std(null_distance_vals)
        
        return (self.evaluate_unnormalized(G_prop) - null_mean)/null_std
    
    
def distance_KS_degree(G0,G_prop,beta=1.):
    """
    Kolmogorov-Smirnov distance between distributions of degrees to the 'beta' power
    """
    return ss.ks_2samp(np.array(G0.vs.degree())**beta,np.array(G_prop.vs.degree())**beta).statistic

    
def null_ER(G0):
    """
    returns random graph with same node and edge counts as G0
    """
    N,M = G0.ecount(),G0.vcount()
    return ig.Graph().Erdos_Renyi(n=N,m=M)

    

class model:
    
    """
    model class
    args:
        -graph generator 'generator' that takes in some named arguments and returns an igraph object
        -'free_parameters' is dict of named parameters to infer posteriors for
        -'priors' is dict of prior classes using same keys as 'free_parameters'
        -'**fixed parameters' is all other named parameters of 'generator' that we fix 
        fixed + free parameters need to fully specify 'generator'
    """
    
    def __init__(self,generator,free_parameters,priors,**fixed_parameters):
        self.generator = generator
        self.fixed_parameters = fixed_parameters
        self.priors = priors
        self.free_parameters = free_parameters
        
        self.free_param_names = list(self.free_parameters.keys())
    
    def prior_sample(self,param):
        """
        sample from prior associated with parameter 'param'
        """
        sample_class = self.priors[param]
        return sample_class.sample()
    
    def generate_graph(self):
        """
        generate graph using draw from prior, which is stored in self.free_parameters 
        """
        for param in self.free_param_names:
            self.free_parameters[param] = self.prior_sample(param)
        return self.generator(**self.free_parameters,**self.fixed_parameters)
        

def generator_PA(N=100,M=3,alpha=1.):
    """
    generalized preferential attachment with exponent alpha, number of nodes and edges N,M
    """
    return ig.Graph().Barabasi(n=N, m=M, power=alpha)


class prior_uniform():
    """
    generate from uniform prior over interval [low,high] using the .sample() method
    """
    def __init__(self,low=0.,high=1.):
        self.low = low
        self.high = high
    
    def sample(self):
        return np.random.uniform()*(self.high-self.low) + self.low

    
class ABC:
    """
    ABC simulator class
    args:
        -'acc': desired acceptance rate
        -'model_object': model object
        -'distance_object': distance object
        -'G0': reference graph to fit for posterior
        -'eps': numerical cutoff value (if applicable)
            -uses acceptance rate if eps == None (default)
        -'hamiltonian': hamiltonian function
            -h(d,eps) is a function of distance d and cutoff scale eps
            -uses hard thresholding hamiltonian by default
    """
    
    def __init__(self,acc,model_object,distance_object,G0,eps='None',hamiltonian='None',\
                 normalized=False,num_nulls='None',null_generator='None',**null_kwargs):
        self.acc = acc
        self.model = model_object
        self.distance = distance_object
        self.G0 = G0
        self.eps = eps
        self.hamiltonian = hamiltonian
        self.normalized = normalized
        self.num_nulls = num_nulls
        self.null_generator = null_generator
        self.null_kwargs  = null_kwargs
        
        self.posterior = dict.fromkeys(self.model.free_param_names)
        self.acceptance_rate = 'None'
        
    def get_eps_from_acc(self,max_runs='None',tol='None',eps_lower_init='None',eps_upper_init='None',\
                         window_size='None',num_sims='None'):
        """
        finds epsilon value that gives approximately the acceptance rate 'acc'
            for the given distance object
        stores this as self.eps
        takes moving average of acceptance rate and tries to get within 'tol' of 'acc'
        updates guess for epsilon by repeatedly bisecting an interval
        args:
            -'eps_init': initial guess for epsilon
            -'max_runs': number of runs algorithm allowed to run for if acc_current
                    never within 'tol' of 'acc'
            -'tol': tolerance of error for accepting acc_current
            -'eps_lower/upper_init': lower/upper cutoff for epsilon bisection
        """
        
        
        eps_currents = []
        for _ in range(num_sims):
            acc_current,run_count = 100,0
            eps_upper = eps_upper_init
            eps_lower = eps_lower_init
            proposal_results = [False]*window_size
            while (run_count < max_runs):

                eps_current = np.random.rand()*(eps_upper-eps_lower) + eps_lower

                for w in range(window_size):

                    G_prop = self.model.generate_graph()
                    params_prop = self.model.free_parameters

                    if self.normalized == True:
                        dist = self.distance.evaluate_normalized(G_prop,num_nulls,null_generator,**null_kwargs)
                    else:
                        dist = self.distance.evaluate_unnormalized(G_prop)

                    if np.log(np.random.rand()) < -self.hamiltonian(dist,eps_current):
                        proposal_results[w] = True
                    else:
                        proposal_results[w] = False

                acc_current = Counter(proposal_results)[True]/window_size

                if abs(acc_current-self.acc) > tol:
                    if acc_current < self.acc:
                        eps_lower = eps_current 
                    else:
                        eps_upper = eps_current
                    eps_current = (eps_upper + eps_lower)/2

                else:
                    break

                #print(run_count,eps_upper,eps_lower,eps_current,acc_current)
                run_count += 1
            eps_currents.append(eps_current)
            
        self.eps = np.mean(eps_currents) 
        
    def run_sims(self,n_runs='None'):
        """
        runs ABC with soft thresholding (hard thresholding imposed by the default hamiltonian h_hard_cutoff)
        n_runs is desired number of runs (proposals) for the simulation
        sets self.accepted_params and self.acceptance_rate
        """
        
        accepted_params = []
        proposal_results = []
        run_count = 0
        while (run_count < n_runs):
                
            G_prop = self.model.generate_graph()
            params_prop = self.model.free_parameters

            if self.normalized == True:
                dist = self.distance.evaluate_normalized(G_prop,num_nulls,null_generator,**null_kwargs)
            else:
                dist = self.distance.evaluate_unnormalized(G_prop)

            if np.log(np.random.rand()) < -self.hamiltonian(dist,self.eps):
                proposal_results.append(True)
                accepted_params.append(params_prop)
            else:
                proposal_results.append(False) 
                    
            run_count += 1 
        
        params = self.model.free_param_names
        posterior_dict = dict.fromkeys(params)
        for p in params:
            posterior_dict[p] = [res[p] for res in accepted_params]
        self.posterior = posterior_dict
        self.acceptance_rate = Counter(proposal_results)[True]/n_runs
    
    def save_to_file(self,filepath,**save_kwargs):
        """
        write to csv with column names as parameter names and column values as the 
            sampled values of that parameter
        """
        try:
            df = pd.DataFrame.from_dict(posterior_dict)
            df.to_csv(filepath,**save_kwargs)
        except:
            print('No posterior data to write!')
        

def h_hard_cutoff(dist,eps):
    """
    hamiltonian with infinite barrier at threshold eps
    """
    if dist < eps:
        return -10
    else:
        return 10
    
N,M=1000,10  
G0 = generator_PA(N,M,alpha=1.3)
distance_obj = distance(G0,distance_KS_degree,beta=1.)
model_obj = model(generator_PA,free_parameters={'alpha':1.},priors={'alpha':prior_uniform(1.,1.5)},N=N,M=M)

In [ ]:
accs = np.linspace(1.,.01,100)
epses = []
for acc in accs:
    print(acc)
    abc_obj = ABC(acc = acc, model_object = model_obj, distance_object = distance_obj, G0 = G0.copy()\
                  , eps = 'None', hamiltonian = h_hard_cutoff, normalized = False, num_nulls = 'None',\
                  null_generator='None')

    abc_obj.get_eps_from_acc(max_runs=15, tol=.0001, eps_lower_init=0., \
                             eps_upper_init=.2, window_size=1000, num_sims=5)
    
    epses.append(abc_obj.eps)

1.0
0.99
0.98
0.97
0.96
0.95
0.94


In [ ]:
import matplotlib.pyplot as plt
plt.scatter(accs,epses)
plt.ylabel('Distance Threshold')
plt.xlabel('Acceptance Rate')
plt.show()

In [46]:
# Discrete choice model with options for:
#         1. Generalized preferential attachment
#             -estimate the exponent
#         2. Triadic closure
#             -estimate the closure probability
#         3. Node fitness (drawn from power law)
#             -estimate the exponent of the fitness distribution
#     Formalism from 'Choosing to Grow a Graph: Modeling Network Formation as Discrete Choice'

        